In [129]:
import pandas as pd

In [130]:
diabet_df=pd.read_csv('data/diabetes_data.csv', sep=',')
#В датасете 778 строк
diabet_df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female
...,...,...,...,...,...,...,...,...,...,...
773,6,103,72,32,190,37.7,0.324,55,0,Female
774,1,71,48,18,76,20.4,0.323,22,0,Female
775,0,117,0,0,0,33.8,0.932,44,0,Female
776,4,154,72,29,126,31.3,0.338,37,0,Female


## Задание 1
Начнём с поиска дубликатов в данных. Найдите все повторяющиеся строки в данных и удалите их. Для поиска используйте все признаки в данных. Сколько записей осталось в данных?

In [131]:
# Список столбцов датасета
diabet_col=list(diabet_df.columns)

# Удаляем дубли, используя для поиска дублей весь набор столбцов
diabet_df=diabet_df.drop_duplicates(subset=diabet_col)

# Смотрим сколько осталось, то есть 10 записей удалено
diabet_df.shape[0]

768

## Задание 2
Далее найдите все неинформативные признаки в данных и избавьтесь от них. В качестве порога информативности возьмите 0.95: удалите все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. В ответ запишите имена признаков, которые вы нашли (без кавычек).

In [132]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabet_df.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabet_df[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabet_df[col].nunique() / diabet_df[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

Gender: 100.0% одинаковых значений


Таким образом, неинформативный столбец у нас один - Gender

In [133]:
# Удаляем неинформативный столбец
diabet_df=diabet_df.drop(columns=['Gender'])

## Задание 3
Попробуйте найти пропуски в данных с помощью метода isnull().

Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!

Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.

Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

In [134]:
diabet_df.isnull()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
763,False,False,False,False,False,False,False,False,False
764,False,False,False,False,False,False,False,False,False
765,False,False,False,False,False,False,False,False,False
766,False,False,False,False,False,False,False,False,False


In [135]:
cols=['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

In [136]:
import numpy as np
for e in cols:
    diabet_df[e]=diabet_df[e].apply(lambda x: np.nan if x==0 else x)

In [137]:
# Считаем долю пропусков по столбцу Insulin
df1=diabet_df.isnull()
d=df1[df1["Insulin"]==True]['Insulin'].count()/df1['Insulin'].count()
d


0.4869791666666667

## Задание 4
Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?

In [138]:
#строим список столбцов с пропусками более 30%
lst=[]
for e in cols:
   d=df1[df1[e]==True][e].count()/df1[e].count()*100
   if d>30:
      lst.append(e) 
      
#удаляем их       
diabet_df=diabet_df.drop(columns=lst)
diabet_df.shape[1]


8

## Задание 5
Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

In [139]:
df=diabet_df.isnull()

In [140]:
diabet_df=diabet_df.dropna(thresh=diabet_df.shape[1]-2, axis=0)
diabet_df

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,35.7,0.148,21,0
2,2,108.0,64.0,NaN,30.8,0.158,21,0
3,8,107.0,80.0,NaN,24.6,0.856,34,0
4,7,136.0,90.0,NaN,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...
763,5,139.0,64.0,35.0,28.6,0.411,26,0
764,1,96.0,122.0,NaN,22.4,0.207,27,0
765,10,101.0,86.0,37.0,45.6,1.136,38,1
766,0,141.0,NaN,NaN,42.4,0.205,29,1


In [141]:
diabet_df.shape[0]

761

## Задание 6
В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых. 

In [142]:
diabet_df

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,35.7,0.148,21,0
2,2,108.0,64.0,NaN,30.8,0.158,21,0
3,8,107.0,80.0,NaN,24.6,0.856,34,0
4,7,136.0,90.0,NaN,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...
763,5,139.0,64.0,35.0,28.6,0.411,26,0
764,1,96.0,122.0,NaN,22.4,0.207,27,0
765,10,101.0,86.0,37.0,45.6,1.136,38,1
766,0,141.0,NaN,NaN,42.4,0.205,29,1


In [143]:
values = {
    'Pregnancies': diabet_df['Pregnancies'].median(),
    'Glucose': diabet_df['Glucose'].median(),
    'BloodPressure': diabet_df['BloodPressure'].median(),
    'SkinThickness': diabet_df['SkinThickness'].median(),
    'BMI': diabet_df['BMI'].median(),
    'DiabetesPedigreeFunction': diabet_df['DiabetesPedigreeFunction'].median(),
    'Age': diabet_df['Age'].median(),
    'Outcome': diabet_df['Outcome'].median()
    
}

In [144]:
diabet_df=diabet_df.fillna(values)

In [145]:
diabet_df['SkinThickness'].mean()

29.109067017082786

## Задание 7
Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?

In [146]:
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [147]:
outliers, cleaned=outliers_iqr(diabet_df, 'SkinThickness')
outliers.shape[0]

87

## Задание 8
Сколько выбросов найдёт классический метод z-отклонения в признаке SkinThickness?
нет ответа



In [160]:
import numpy as np

In [161]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [162]:
outliers, cleaned=outliers_z_score(diabet_df, 'SkinThickness')
outliers.shape[0]

4

## Задание 9
На приведённой гистограмме показано распределение признака DiabetesPedigreeFunction. Такой вид распределения очень похож на логнормальный, и он заставляет задуматься о логарифмировании признака. Найдите сначала число выбросов в признаке DiabetesPedigreeFunction с помощью классического метода межквартильного размаха.

Затем найдите число выбросов в этом же признаке в логарифмическом масштабе (при логарифмировании единицу прибавлять не нужно!). Какова разница между двумя этими числами (вычтите из первого второе)?

In [181]:
def outliers_iqr_log(data, feature,log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    print(iqr)
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

In [182]:
# Без логарифма
outliers1, cleaned1=outliers_iqr_log(diabet_df, 'DiabetesPedigreeFunction', log_scale=False)
k1=outliers1.shape[0]
# С логарифмом
outliers2, cleaned2=outliers_iqr_log(diabet_df, 'DiabetesPedigreeFunction', log_scale=True)
k2=outliers2.shape[0]


0.385
0.9469630314072386


In [184]:
k=k2-k1
k

-29